In [5]:
import os
import pandas as pd
import requests
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
from dotenv import load_dotenv

import time

In [6]:
load_dotenv()

host = os.environ.get("snowflake_host")
user = os.environ.get("snowflake_user")
password = os.environ.get("snowflake_password")
account = os.environ.get("snowflake_account")
warehouse = os.environ.get("snowflake_warehouse")
database = os.environ.get("snowflake_database")
schema = os.environ.get("snowflake_schema")

query = "select * from DBT_PROJECT03.stg__transit_data"
connector = snowflake.connector.connect(
    host=host,
    user=user,
    password=password,
    account=account,
    warehouse=warehouse,
    database=database,
    schema=schema)

try:
    cursor = connector.cursor().execute(query)
finally:
    connector.close()

df = cursor.fetch_pandas_all()
df['COORD'] = df[['LAT', 'LONG']].astype(str).apply(','.join, axis=1)

df.head()

,CODE,NAME,LAT,LONG,COORD
0,G1,Krung Thon Buri,13.72083,100.50278,"13.72083,100.50278"
1,G2,Charoen Nakhon,13.72650,100.50900,"13.7265,100.509"
2,G3,Khlong San,13.73040,100.50760,"13.7304,100.5076"
3,PP02,Talad Bang Yai,13.88096,100.40924,"13.88096,100.40924"
4,PP01,Khlong Bang Phai,13.89242,100.40827,"13.89242,100.40827"


In [7]:
def query_places(code, location):

    places_data = []
    df_temp = None
    url = "https://api.foursquare.com/v3/places/search"
    api_key = os.environ.get("places_api_key")

    params = {
        "ll": location,
        "sort":"DISTANCE",
        "radius":"100000",
        "categories":"10000,13000,17000",
        "limit":"50"
    }

    headers = {
        "Accept": "application/json",
        "Authorization": api_key
    }

    response = requests.request("GET", url, params=params, headers=headers)

    if response.status_code == 200: 
        places_data.append(response.json())
        df_temp = pd.json_normalize(places_data, record_path=['results'])
        df_temp.drop(columns=['categories'], inplace=True)

        df_extract = df_temp[['fsq_id', 'name', 'geocodes.main.latitude', 'geocodes.main.longitude', 'location.formatted_address']]

        os.makedirs('data', exist_ok=True)
        df_extract.to_csv(f'data/{code}_places.csv', index=False)

    return df_extract

In [8]:
df['RESULT'] = df.apply(lambda x: query_places(x['CODE'], x['COORD']), axis=1)